## Subagents
Multi-agent system 중 Subagents는 Supervisor Agent가 하위 Agent (Sub-agent) 들을 관리하는 구조를 의미합니다.

**요청 흐름도** <br>
User -> Supervisor Agent -> Sub-agent -> Supervisor Agent -> User

**튜토리얼 진행** <br>
이 노트북에서는 Supervisor Pattern 기반 개인 비서 에이전트를 구축해보는 튜토리얼을 진행합니다.<br>

- 비서 에이전트: Supervisor 에이전트 입니다. Sub-agent 로 캘린더 에이전트와 이메일 에이전트를 두고 있습니다. 
- 캘린더 에이전트: 스케쥴을 관리하고, 체크하고 일정을 관리합니다.
- 이메일 에이전트: 소통을 담당하고 메세지를 생성해서 알림을 보냅니다.

튜토리얼 진행은 아래와 같습니다.
1. 환경설정
2. 도구 정의
3. Sub-agent 정의
4. Sub-agent 를 도구로 래핑
5. Supervisor Agent 정의
6. (고급) Human-in-the-loop: 승인 절차 추가
 


### 1. 환경설정

In [1]:
# !pip install -U langchain langchain-openai langgraph python-dotenv

In [2]:
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
import os

load_dotenv()

openai_key = os.environ["OPENAI_API_KEY"]
langsmith_key = os.environ["LANGSMITH_API_KEY"]

model = init_chat_model("gpt-4o-mini")

print(openai_key[:5], "...")  # 디버깅용


sk-pr ...


### 2. 도구 정의

에이전트들이 사용할 도구를 미리 정의합니다. <br>
tool 은 함수에 어노테이션을 붙여서 사용합니다. 

In [3]:
from langchain.tools import tool

@tool
def create_calendar_event(
    title: str,
    start_time: str,      
    end_time: str,         
    attendees: list[str],
    location: str = ""
) -> str:
    """캘린더 이벤트를 생성합니다. 반드시 ISO 날짜 형식을 사용해야 합니다."""
    return f"이벤트 생성 완료: {title} ({start_time} ~ {end_time}), 참석자: {len(attendees)}명"

@tool
def send_email(
    to: list[str],
    subject: str,
    body: str,
    cc: list[str] = []
) -> str:
    """이메일을 발송합니다."""
    return f"이메일 발송 완료: To: {', '.join(to)} - 제목: {subject}"

@tool
def get_available_time_slots(
    attendees: list[str],
    date: str,     
    duration_minutes: int
) -> list[str]:
    """특정 날짜의 참석 가능한 시간대를 확인합니다."""
    return ["09:00", "14:00", "16:00"]

### 3. Sub-agent 정의
위에서 정의했던 도구들은 creat_agent 메서드의 tools 속성에서 사용할 수 있습니다. <br>
어떤 하나의 agent 를 정의할땐 결국 **model, tool, system_prompt**  이 세가지가 기본입니다.
<br> 
Tips
- 시스템 프롬프트는 맥락과 파라미터에 대한 내용과 규칙을 적어주는 것이 좋습니다.
- 결국엔 도구를 호출할때도 AI 가 알아서 파라미터값을 넣고 호출합니다. 따라서 이값을 LLM이 도구 목적에 맞게 잘 넣는것이 중요합니다.
- 시스템 프롬프트에는 도구의 이름을 명확히 적어주는것이 좋습니다.


In [4]:
from langchain.agents import create_agent

CALENDAR_AGENT_PROMPT = (
    "당신은 일정 관리를 돕는 캘린더 스케줄링 어시스턴트입니다. "
    "‘다음 주 화요일 오후 2시’와 같은 자연어 일정 요청을 "
    "ISO 형식의 날짜·시간(datetime)으로 정확히 변환하세요. "
    "필요한 경우 get_available_time_slots를 사용해 가능한 시간대를 확인하세요. "
    "일정을 등록할 때는 create_calendar_event를 사용하세요. "
    "최종 응답에서는 실제로 어떤 일정이 등록되었는지 반드시 확인해 주세요."
)


calendar_agent = create_agent(
    model,
    tools=[create_calendar_event, get_available_time_slots],
    system_prompt=CALENDAR_AGENT_PROMPT,
)

In [5]:

query = "팀미팅을 오후 2시부터 2시간 잡아줘"
 
for step in calendar_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_available_time_slots (call_l4DnM37yGwNeonzoDQkzWjYe)
 Call ID: call_l4DnM37yGwNeonzoDQkzWjYe
  Args:
    attendees: ['팀원']
    date: 2023-10-31
    duration_minutes: 120
================================= Tool Message =================================
Name: get_available_time_slots

["09:00", "14:00", "16:00"]
================================== Ai Message ==================================
Tool Calls:
  create_calendar_event (call_HsR1CFrOfPzdgtf1CttKKwqP)
 Call ID: call_HsR1CFrOfPzdgtf1CttKKwqP
  Args:
    title: 팀미팅
    start_time: 2023-10-31T14:00:00
    end_time: 2023-10-31T16:00:00
    attendees: ['팀원']
================================= Tool Message =================================
Name: create_calendar_event

이벤트 생성 완료: 팀미팅 (2023-10-31T14:00:00 ~ 2023-10-31T16:00:00), 참석자: 1명
================================== Ai Message ==================================

일정이 등록되었습니다. 

- **회의 제목**

위 메세지를 분해해보면 AI 가 tool call 2번에 거쳐 최종응답을 생성했습니다! <br>
파라미터는 시스템 프롬프트에 따라 알아서 잘 넣는걸 볼 수 있습니다. <br>
참석자는 적지 않았는데 참석자를 적어놨네요...<br>
똑같이 이메일 에이전트로 만들겠습니다.

In [6]:
EMAIL_AGENT_PROMPT = (
    "당신은 이메일 작성을 돕는 이메일 어시스턴트입니다. "
    "자연어로 된 요청을 바탕으로 전문적인 이메일을 작성하세요. "
    "수신자 정보를 추출하고, 적절한 제목과 본문 내용을 구성하세요. "
    "이메일을 발송할 때는 send_email을 사용하세요. "
    "최종 응답에서는 실제로 어떤 이메일이 발송되었는지 반드시 확인해 주세요."
)


email_agent = create_agent(
    model,
    tools=[send_email],
    system_prompt=EMAIL_AGENT_PROMPT,
)

In [7]:
query = "디자인 팀에게 새 목업을 검토해 달라는 알림을 보내세요"

for step in email_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  send_email (call_a7FxUt1QJ5WjkHHkfshbClwB)
 Call ID: call_a7FxUt1QJ5WjkHHkfshbClwB
  Args:
    to: ['designteam@example.com']
    subject: 새 목업 검토 요청
    body: 안녕하세요 디자인 팀, 

새로운 목업이 준비되었습니다. 검토해 주시기 바랍니다. 피드백은 이번 주 금요일까지 부탁드립니다.

감사합니다.
================================= Tool Message =================================
Name: send_email

이메일 발송 완료: To: designteam@example.com - 제목: 새 목업 검토 요청
================================== Ai Message ==================================

디자인 팀에게 "새 목업 검토 요청"이라는 제목으로 이메일이 성공적으로 발송되었습니다. 이메일 본문 내용은 다음과 같습니다:

```
안녕하세요 디자인 팀,

새로운 목업이 준비되었습니다. 검토해 주시기 바랍니다. 피드백은 이번 주 금요일까지 부탁드립니다.

감사합니다.
```

추가 도움이 필요하시면 말씀해 주세요!


캘린더 에이전트와 마찬가지로 잘하네요

### 4. Sub-agent 를 도구로 래핑
Sub-agent 를 이제 도구로 래핑을 해서 Supervisor-agent 가 사용할 수 있도록 만들어주겠습니다.<br>
말이 좀 그렇지만 Supervisor-agent 는 Sub-agent 를 그냥 도구처럼 사용하는 것입니다. <br>
또한 계층적이기 때문에 Supervisor-agent 는 Sub-agent 가 무슨 도구를 들고있는지, 어떤 일을 하는지 알지 못합니다. <br>
그저 Sub-agent 의 이름, 요청 파라미터, 설명만 알고 있습니다.

In [8]:
@tool
def schedule_event(request: str) -> str:
    """자연어를 사용해 캘린더 일정을 관리합니다.

    사용자가 일정을 생성, 수정하거나 확인하고자 할 때 사용하세요.
    날짜/시간 파싱, 가능 시간대 확인, 일정 생성까지 처리합니다.

    입력값: 자연어 일정 요청
    (예: '다음 주 화요일 오후 2시에 디자인 팀과 미팅')
    """
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text


@tool
def manage_email(request: str) -> str:
    """자연어를 사용해 이메일을 발송합니다.

    알림, 리마인더 등 이메일 전송이 필요한 경우 사용하세요.
    수신자 정보 추출, 제목 생성, 이메일 본문 작성을 처리합니다.

    입력값: 자연어 이메일 요청
    (예: '회의에 대한 리마인더를 보내줘')
    """
    result = email_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text


### 5. Supervisor Agent 정의
이제 주인님을 만듭시다. 비서 에이전트는 도구를 2개 들고 있습니다 (슬프지만 이게 Sub-agent 인거죠).<br>


In [9]:
SUPERVISOR_PROMPT = (
    "당신은 사용자를 돕는 개인 비서입니다. "
    "캘린더 일정을 등록하고 이메일을 발송할 수 있습니다. "
    "사용자의 요청을 적절한 도구 호출로 분해하고, 결과를 종합하여 처리하세요. "
    "요청에 여러 작업이 포함된 경우, 여러 도구를 순차적으로 사용하세요."
)

supervisor_agent = create_agent(
    model,
    tools=[schedule_event, manage_email],
    system_prompt=SUPERVISOR_PROMPT,
)

In [10]:
query = "내일 오전 9시에 팀 스탠드업 미팅을 일정에 등록해줘"

for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_Yra2HImVkhqw1jzRHAMW2FCd)
 Call ID: call_Yra2HImVkhqw1jzRHAMW2FCd
  Args:
    request: 내일 오전 9시에 팀 스탠드업 미팅
================================= Tool Message =================================
Name: schedule_event

일정이 등록되었습니다: 

- **이벤트 제목**: 팀 스탠드업 미팅
- **시작 시간**: 2023-10-30T09:00:00
- **종료 시간**: 2023-10-30T09:30:00
- **참석자**: 0명

필요한 경우 추가적인 참석자를 등록해 드릴 수 있습니다.
================================== Ai Message ==================================

팀 스탠드업 미팅이 내일 오전 9시에 일정에 등록되었습니다. 

- **이벤트 제목**: 팀 스탠드업 미팅
- **시작 시간**: 2023-10-30T09:00:00
- **종료 시간**: 2023-10-30T09:30:00

필요한 경우 참석자를 추가로 등록할 수 있습니다. 추가적인 요청이 있으신가요?


In [11]:
query = (
    "다음 주 화요일 오후 2시에 디자인 팀과 1시간짜리 회의를 일정에 등록하고, "
    "새 목업을 검토해 달라는 이메일 리마인드를 보내줘."
)


for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_zVAZjwSZsVAG67aXd47abwoR)
 Call ID: call_zVAZjwSZsVAG67aXd47abwoR
  Args:
    request: 다음 주 화요일 오후 2시에 디자인 팀과 1시간짜리 회의
  manage_email (call_pXubHIQkb69QMhQQyPkABHhV)
 Call ID: call_pXubHIQkb69QMhQQyPkABHhV
  Args:
    request: 디자인 팀에게 새 목업을 검토해 달라는 리마인더 이메일
================================= Tool Message =================================
Name: schedule_event

일정이 성공적으로 등록되었습니다. 

**일정 내용:**
- **제목:** 디자인 팀 회의
- **날짜 및 시간:** 2023년 10월 31일 오후 2시 ~ 오후 3시
- **참석자:** 디자인 팀

필요한 다른 일정 조정이나 요청이 있다면 말씀해 주세요!
================================= Tool Message =================================
Name: manage_email

디자인 팀에게 새 목업을 검토해 달라는 리마인더 이메일이 성공적으로 발송되었습니다. 내용은 다음과 같습니다:

- **수신자**: design_team@example.com
- **제목**: 리마인더: 새 목업 검토 요청
- **본문**:
  ```
  안녕하세요, 디자인 팀 여러분.

  새 목업을 검토해 주실 것을 리마인드 드립니다. 최종 피드백을 바탕으로 프로젝트를 진행해야 할 시점이 다가오고 있습니다. 검토 후, 의견을 주시면 감사하겠습니다.

  감사합니다.

  [당신의 이름]
  

### 6. (고급) Human-in-the-loop: 승인 절차 추가
langchain 에는 Agent 가 도구를 사용 중 중간에 동작을 끼워넣을 수 있습니다. <br>
도구의 응답을 정리한다거나, 사람이 개입해서 승인한다거나 하는 방식으로 할 수 있습니다. <br>
응답이 interupt 로 온다면 백엔드단에서 사람의 승인을 받거나, 여러 선택지의 도구를 호출하는 로직을 만들어 볼 수도 있겠네요!

In [12]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware 
from langgraph.checkpoint.memory import InMemorySaver 


calendar_agent = create_agent(
    model,
    tools=[create_calendar_event, get_available_time_slots],
    system_prompt=CALENDAR_AGENT_PROMPT,
    middleware=[ 
        HumanInTheLoopMiddleware( 
            interrupt_on={"create_calendar_event": True}, 
            description_prefix="캘린더 일정 생성 승인 대기 중",
        ), 
    ], 
)

email_agent = create_agent(
    model,
    tools=[send_email],
    system_prompt=EMAIL_AGENT_PROMPT,
    middleware=[ 
        HumanInTheLoopMiddleware( 
            interrupt_on={"send_email": True}, 
            description_prefix="이메일 발송 승인 대기 중",
        ), 
    ], 
)

supervisor_agent = create_agent(
    model,
    tools=[schedule_event, manage_email],
    system_prompt=SUPERVISOR_PROMPT,
    checkpointer=InMemorySaver(), 
)

In [ ]:
query = (
    "다음 주 화요일 오후 2시에 디자인 팀과 1시간 회의를 일정에 등록하고, "
    "새 목업을 검토해 달라는 이메일 리마인드를 보내줘."
)

config = {"configurable": {"thread_id": "6"}}

interrupts = []
for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    config,
):
    for update in step.values():
        if isinstance(update, dict):
            for message in update.get("messages", []):
                message.pretty_print()
        else:
            interrupt_ = update[0]
            interrupts.append(interrupt_)
            print(f"\nINTERRUPTED: {interrupt_.id}")
            

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_DydTs3fKyzUFAkc24Pm95ZV1)
 Call ID: call_DydTs3fKyzUFAkc24Pm95ZV1
  Args:
    request: 다음 주 화요일 오후 2시에 디자인 팀과 미팅, 1시간 진행
  manage_email (call_Wn18IRvHg3q9mlAj81TzPKZC)
 Call ID: call_Wn18IRvHg3q9mlAj81TzPKZC
  Args:
    request: 디자인 팀에게 새로운 모형을 검토하라는 이메일 리마인더 보내줘

INTERRUPTED: e072b7c704489b764baf68be40001695

INTERRUPTED: 2f36f24cd6574f079573bcf8778b0650


In [15]:
for interrupt_ in interrupts:
    for request in interrupt_.value["action_requests"]:
        print(f"INTERRUPTED: {interrupt_.id}")
        print(f"{request['description']}\n")

INTERRUPTED: e072b7c704489b764baf68be40001695
이메일 발송 승인 대기 중

Tool: send_email
Args: {'to': ['design_team@example.com'], 'subject': '리마인더: 새로운 모형 검토 요청', 'body': '안녕하세요 디자인 팀,\n\n새로운 모형에 대한 검토가 아직 완료되지 않은 것 같아 리마인더드립니다. 검토 결과와 피드백을 다음 회의 전까지 주시면 감사하겠습니다.\n\n감사합니다.\n\n[당신의 이름]'}

INTERRUPTED: 2f36f24cd6574f079573bcf8778b0650
캘린더 일정 생성 승인 대기 중

Tool: create_calendar_event
Args: {'title': '디자인 팀과 미팅', 'start_time': '2023-10-31T14:00:00', 'end_time': '2023-10-31T15:00:00', 'attendees': ['디자인 팀']}



### 요약

Subagent 구축시 중요한 개념 2개를 정리해보자면
1. **도구 래핑 (Tool Wrapping)**

Supervisor가 하위 에이전트의 세부 구현이나 개별 도구들을 알 필요 없이, <br> 
각 에이전트를 하나의 독립적인 '도구'로 취급하여 호출할 수 있도록 래핑하여 계층적 구조를 완성합니다. <br>

2. **파라미터 전달 (Parameter Passing)**

Supervisor는 사용자의 복잡한 자연어 요청을 분석하여, 래핑된 하위 에이전트가 <br>
업무를 수행하는 데 필요한 핵심 파라미터(일정 시간, 수신자, 메일 본문 등)를 정확하게 추출하여 전달하는게 중요합니다.
